In [1]:
from docx import Document
import pypandoc
import pandas as pd
import docx.document
import docx.oxml.table
import docx.oxml.text.paragraph
import docx.table
import docx.text.paragraph
import re
import nltk
from nltk.corpus import stopwords 
import joblib
import copy
import warnings
warnings.filterwarnings("ignore")
from docx.oxml.text.paragraph import CT_P
from docx.text.paragraph import Paragraph
from docx.oxml.xmlchemy import OxmlElement
import re
import os
import win32com.client as win32
from win32com.client import constants
from tqdm.notebook import tqdm_notebook

In [2]:
def save_as_docx(path):
    """
    Convert .doc files to .docx files
    """
   # Opening MS Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    doc = word.Documents.Open(path)
    doc.Activate()

    # Rename path with.docx
    new_file_abs = os.path.abspath(path)
    new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)

    # Save and Close
    word.ActiveDocument.SaveAs(
       new_file_abs, FileFormat = constants.wdFormatXMLDocument
    )
    doc.Close(False)

In [3]:
def save_as_doc(path):
    """
    Convert .docx files to .doc files
    """
   # Opening MS Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    docx = word.Documents.Open(path)
    docx.Activate()

    # Rename path with.docx
    new_file_abs = os.path.abspath(path)
    new_file_abs = re.sub(r'\.\w+$', '.doc', new_file_abs)

    # Save and Close
    word.ActiveDocument.SaveAs(
       new_file_abs, FileFormat = constants.wdFormatXMLDocument
    )
    docx.Close(False)

In [4]:
def iter_paragraphs(parent, recursive=True):
    """
    Yield each paragraph and table child within *parent*, in document order.
    Each returned value is an instance of Paragraph. *parent*
    would most commonly be a reference to a main Document object, but
    also works for a _Cell object, which itself can contain paragraphs and tables.
    Images are inline objects of Paragraphs.
    """
    if isinstance(parent, docx.document.Document):
        parent_elm = parent.element.body
    elif isinstance(parent, docx.table._Cell):
        parent_elm = parent._tc
    else:
        raise TypeError(repr(type(parent)))

    for child in parent_elm.iterchildren():
        if isinstance(child, docx.oxml.text.paragraph.CT_P):
            yield docx.text.paragraph.Paragraph(child, parent)
        elif isinstance(child, docx.oxml.table.CT_Tbl):
            if recursive:
                table = docx.table.Table(child, parent)
                yield table
        else:
            print(child)


In [5]:
def feature_extraction(df, drop_section = True):
    """
    Extracts useful information in a text message. Used internally in modifying the document
    """
    df_new=df[['extracted_text','contains_image','contains_table']]
    df_new.reset_index(drop=True, inplace=True)
    df_new['extracted_text'] = df_new['extracted_text'].str.strip()
    import nltk
    from nltk.corpus import stopwords  
    df_new['words'] = df_new['extracted_text'].apply(lambda x: len(x.split()))
    stop_words = set(stopwords.words('english'))
    df_new['stopwords'] = df_new['extracted_text'].apply(lambda x: len(set(x.split()) & stop_words))
    df_new['endswithdot']=df_new['extracted_text'].str.endswith('.')
    df_new['endswithdot']=df_new['endswithdot'].astype(int)
    df_new['Table']=df_new['extracted_text'].str.contains('Table')
    df_new['Table']=df_new['Table'].astype(int)
    df_new['contains section']=df_new['extracted_text'].str.contains('Section')
    df_new['contains section']=df_new['contains section'].astype(int)
    df_new['contains percentage']=df_new['extracted_text'].str.contains('%')
    df_new['contains percentage']=df_new['contains percentage'].astype(int)
    df_new['Uppercase'] = df_new['extracted_text'].str.findall(r'[A-Z]').str.len()
    df_new['Lowercase'] = df_new['extracted_text'].str.findall(r'[a-z]').str.len()
    df_new['percentage capital']=(df_new['Uppercase']*100)/(df_new['Lowercase']+df_new['Uppercase'])
    if drop_section:
        df_new.drop(columns=['Uppercase','Lowercase','contains section'],inplace=True)
    else:
        df_new.drop(columns=['Uppercase', 'Lowercase'], inplace=True)
    import re
    def fun_starts_integer(x):
        reg = r"\d+?\s.*"
        pat  = re.compile(reg)
        if pat.match(x)==None:
            return 0
        else:
            return 1
    def fun_starts_decimal(x):
        reg = r"\d+\.\d+?\s.*"
        pat  = re.compile(reg)
        if pat.match(x)==None:
            return 0
        else:
            return 1
    df_new['starts_integer']=df_new['extracted_text'].apply(lambda x:fun_starts_integer(x))
    df_new['starts_decimal']=df_new['extracted_text'].apply(lambda x:fun_starts_decimal(x))
    df_new.fillna(0,inplace=True)
    return df_new


In [6]:
def insert_paragraph_before(item, text, style=None):
    """
    Return a newly created paragraph, inserted directly before this
    item (Table, etc.).
    """
    p = CT_P.add_p_before(item._element)
    p2 = Paragraph(p, item._parent)
    p2.text = text
    p2.style = style
    return p2

In [7]:
def insert_paragraph_after(item, text, style=None):
    """
    Return a newly created paragraph, inserted directly before this
    item (Table, etc.).
    Used internally in update_Document function.
    """
    new_p = OxmlElement("w:p")
    item._element.addnext(new_p)
    new_para = Paragraph(new_p, item._parent)
    new_para.text = text
    new_para.style = style
    return new_para

In [8]:
def delete_paragraph(paragraph):
    """
    Deletes the paragraph that is given as reference.
    Used Internally in update_Document
    """
    p = paragraph._element
    p.getparent().remove(p)
    p._p = p._element = None

In [9]:
startTags = ["<<START_HEADING>>","<<START_SUBHEADING>>","<<START_SECTION>>"]
endTags = ["<<END_HEADING>>","<<END_SUBHEADING>>","<<END_SECTION>>"]
clf = joblib.load(open("random_forest_vfinal.pkl", "rb")) 
def update_Document(document2):
    """
    Adds start and endtags to paragraphs in docx file 
    predicted using trained classifier.
    Takes care of merging consecutive sections,
    handling tables etc.
    """
    document = copy.deepcopy(document2)
    prev_para = None
    prev_isTable = False
    for curr in tqdm_notebook(iter_paragraphs(document)):
        image = 0
        table = 0
        hyperlink = 0
        if isinstance(curr,docx.text.paragraph.Paragraph):
            text = curr.text
            if 'graphicData' in curr._p.xml:
                image = 1 
            if curr._p.xpath("./w:hyperlink"):
                hyperlink = 1
        elif isinstance(curr,docx.table.Table):
            table = 1 
            text = ""
            for row in curr.rows:
                for cell in row.cells:
                    temp = cell.text
                    temp = temp.replace('\n','')
                    text+=temp 

        df = pd.DataFrame({'extracted_text': [text],'contains_image': [image],'contains_table':[table]})
        df = feature_extraction(df)
        df = df.drop(['extracted_text'],axis=1)
        df = df.loc[~(df==0).all(axis=1)]
        try: # try and except block will take care of blank lines
            if clf.predict(df)[0] ==0 or clf.predict(df)[0] == 1:
                if isinstance(curr,docx.text.paragraph.Paragraph) and hyperlink==0 and image==0:
                    curr.text = startTags[clf.predict(df)[0]] + curr.text + endTags[clf.predict(df)[0]]
                elif isinstance(curr,docx.table.Table) or hyperlink==1 or image==1:
                    insert_paragraph_before(curr,startTags[clf.predict(df)[0]])
                    insert_paragraph_after(curr,endTags[clf.predict(df)[0]])
                prev_para = None
                prev_isTable = False

            else:
                if prev_para == None:
                    if isinstance(curr,docx.text.paragraph.Paragraph) and hyperlink==0 and image==0:
                        curr.text = startTags[clf.predict(df)[0]] + curr.text
                        curr.add_run(endTags[clf.predict(df)[0]])
                        prev_para = curr 
                        prev_isTable = False 
                    elif isinstance(curr,docx.table.Table) or hyperlink==1 or image==1:
                        insert_paragraph_before(curr,startTags[clf.predict(df)[0]])
                        prev_para = insert_paragraph_after(curr,endTags[clf.predict(df)[0]])
                        prev_isTable = True 

                elif prev_para:
                    if prev_isTable == False:
                        if endTags[clf.predict(df)[0]] in prev_para.text:
                            inline = prev_para.runs
                            for i in range(len(inline)):
                                if endTags[clf.predict(df)[0]] in inline[i].text:
                                    text_ = inline[i].text.replace(endTags[clf.predict(df)[0]], '')
                                    inline[i].text = text_
                                    
                        
                    elif prev_isTable:
                        delete_paragraph(prev_para)
                        prev_para = None 

                    if isinstance(curr,docx.text.paragraph.Paragraph) and hyperlink==0 and image==0:
                        curr.add_run(endTags[clf.predict(df)[0]])
                        prev_para = curr 
                        prev_isTable = False 

                    elif isinstance(curr,docx.table.Table) or hyperlink==1 or image==1:
                        prev_para = insert_paragraph_after(curr,endTags[clf.predict(df)[0]])
                        prev_isTable = True 
                
                    
        except:
            pass
    return document

In [15]:
clf_ = joblib.load(open("best_svc_txt.pkl", "rb"))
def update_text(location):
    startTags = ["<<START_HEADING>>","<<START_SUBHEADING>>","<<START_SECTION>>"]
    endTags = ["<<END_HEADING>>","<<END_SUBHEADING>>","<<END_SECTION>>"]
    document = open(location,'r')
    filename, file_extension = os.path.splitext(location)
    new_doc = open(filename + 'AstraZeneca Submission' + file_extension,'w')
    flag = 0
    for text in tqdm_notebook(document.readlines()):
        if 'Figure' in text:
            image = 1 
        else:
            image = 0
        if 'Table' in text:
            table = 1 
        else:
            table = 0 
        text_ = text.rstrip()
        df = pd.DataFrame({'extracted_text': [text_],'contains_image': [image],'contains_table':[table]})
        df = feature_extraction(df)
        df = df.drop(['extracted_text'],axis=1)
        df = df.loc[~(df==0).all(axis=1)]
        try:
            if clf_.predict(df)[0] ==0 or clf_.predict(df)[0] == 1:
                if flag == 0:
                    temp_text = startTags[clf_.predict(df)[0]] + text + endTags[clf_.predict(df)[0]] + '\n'
                    new_doc.write(temp_text)
                elif flag ==1:
                    flag = 0
                    new_doc.write('<<END_SECTION>>\n')
                    temp_text = startTags[clf_.predict(df)[0]] + text + endTags[clf_.predict(df)[0]] + '\n'
                    new_doc.write(temp_text)
                    
            else:
                if flag ==0:
                    flag = 1 
                    temp_text = startTags[clf_.predict(df)[0]] + text
                    new_doc.write(temp_text)
                elif flag ==1:
                    temp_text = text 
                    new_doc.write(temp_text)
        except Exception as e:
            new_doc.write('\n')
        
    if flag ==1:
        new_doc.write('<<END_SECTION>>')
        flag = 0
    
    new_doc.close()

## Input File

In [11]:
# enter path to input files
# enter full paths (preferably in backslash way)
path1 = r'D:\Sem7-Vacation\AZ\submission\data\input-word-document.docx'
path2 = r'D:\Sem7-Vacation\AZ\submission\data\input-word-document_file.doc' 
path3 = r'D:\Sem7-Vacation\AZ\submission\data\input-plain-document.txt'


In [12]:
def convert_file(path):
    filename, file_extension = os.path.splitext(path) 
    
    if file_extension == '.docx':
        input_doc = Document(path)
        out = update_Document(input_doc) 
        out.save(filename+'AstraZeneca_Submission 1.docx')
        
    elif file_extension == '.doc':
        try:
            save_as_docx(path)
            path_ = path+'x'
            input_doc = Document(path_)
            out = update_Document(input_doc)
            outputfile = filename+ 'AstraZeneca_Submission 2.docx'
            out.save(outputfile)
            save_as_doc(outputfile)
            os.remove(outputfile)
            os.remove(path_)
        except Exception as e:
            print('Failed to open .doc extension. ',e)
            
    elif file_extension == '.txt':
        update_text(path)     
    
    else:
        print('Invalid File Type')

In [13]:
convert_file(path1)

<CT_SectPr '<w:sectPr>' at 0x1e40de75ae8>



In [32]:
convert_file(path2)

<CT_SectPr '<w:sectPr>' at 0x23dd9c96138>



In [16]:
convert_file(path3)